In [5]:
import pandas as pd
import os 
import sys
import numpy as np
import sklearn
import time
import copy
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import SubsetRandomSampler 
import torch.nn.functional as F
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
#create single pose dataset for dnn model
class SinglePoseDataset(Dataset):
    
    def __init__(self, n_frames=5):
        
        # get data and label from csvget 
        dataset, labels = SinglePoseDataset.ReadPoseData(n_frames)
        
        self.X = dataset
        self.y = labels
    
    # read dataset from csv file
    @staticmethod
    def ReadPoseData(n_frames):
        
        # get csv file path
        curr_dir = os.getcwd()
        csv_file_path = os.path.join(curr_dir, 'data/drop_res.csv')
        
        # list for storing data and labels
        data  = []
        label = []
        
        # lenth of sequence
        #n_frames = SinglePoseDataset.n_frames
        
        # read csv file
        KP_df = pd.read_csv(csv_file_path)
        
        # convert pos_class to categories
        KP_df['pos_class'] = KP_df['pos_class'].astype('category')
        KP_df['pos_class'] = KP_df['pos_class'].cat.codes

        # skipping (0-3) colomns , return values of all rows and columns from 4 to last
        features = KP_df.iloc[:,5:].values
        #return values of pose_class 
        pose_class = KP_df['pos_class'].values
        # normalize keypoints 
        SinglePoseDataset.normalize_min_(features)
        # append multiple rows to create a sequence of data
        for i in range(features.shape[0]-n_frames):
            data.append(features[i:i+n_frames,...])
            label_sequence = pose_class[i:i+n_frames]
            unique, counts = np.unique(label_sequence, return_counts=True)
            label.append(unique[np.argmax(counts)])
            
        data , label =  np.array(data, dtype = np.float), np.array(label, dtype = np.int_)
        
        return data , label
    
    # min-max normalization to scale the x, y coordinates in range (0-1) 
    @staticmethod
    def normalize_min_(pose:np.ndarray):
        pose = pose.reshape(len(pose),-1,2)
        for i in range(len(pose)):
            xmin = np.min(pose[i,:,0]) 
            ymin = np.min(pose[i,:,1])
            xlen = np.max(pose[i,:,0]) - xmin
            ylen = np.max(pose[i,:,1]) - ymin

            if(xlen==0): pose[i,:,0]=0
            else:
                pose[i,:,0] -= xmin 
                pose[i,:,0] /= xlen

            if(ylen==0): pose[i,:,1]=0
            else:
                pose[i,:,1] -= ymin
                pose[i,:,1] /= ylen
        return pose
    
    # number of rows in the dataset
    def __len__(self):
        
        return len(self.X)
        
    # get a row at an index
    def __getitem__(self, idx):
        
        data  = torch.tensor(self.X[idx], dtype=torch.float) 
        label = torch.tensor(self.y[idx], dtype=torch.long)
        
        return [data,label]
    
    # get indexes for train and test rows
    
    '''
    def get_splits(self, n_test = 0.2, n_valid=0.2):
        
        # determine sizes 
        test_size = round(n_test * len(self.X))
        valid_size = round(n_valid * len(self.X))
        train_size = len(self.X)-(test_size+valid_size)
        print(train_size, valid_size, test_size)
        # calculate the split 
        return random_split(self, [train_size, valid_size, test_size])
    '''
    
    def get_class_labels(self):
        
        labels = ["Fall","Stand", "Tie"]
        
        return labels
    
    def reshape_features(self):
        self.X = self.X.reshape(-1, self.X.shape[1]*self.X.shape[2])

In [3]:
dataset = SinglePoseDataset(n_frames=20)
targets = dataset.y

In [4]:
train_idx, valid_idx = train_test_split(np.arange(len(targets)), test_size=0.2, stratify=targets)

In [ ]:
train_idx[1:10]

In [ ]:
np.unique(targets[train_idx], return_counts=True)

In [ ]:
np.unique(targets[val_idx], return_counts=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dataset.X, dataset.y, test_size=0.2, stratify=dataset.y)
np.unique(y_train, return_counts=True)
np.unique(y_val, return_counts=True)

In [ ]:
train_loader = DataLoader(X_train,)

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(val_idx)
train_loader  = DataLoader(dataset, batch_size=32, sampler=train_sampler, drop_last=True)
valid_loader  = DataLoader(dataset, batch_size=32, sampler=valid_sampler, drop_last=True)

In [ ]:
train_loader.sampler.__len__()

In [ ]:
np.unique(dataset.y, return_counts=True)

### Tips and tricks to training Fall Models
* FallNet Architecture:
* Input ---> (FC:1024 --> batch_norm --> ReLU --> Dropout)*2 + input --> output 
* Output ---> (FC:1024 --> batch_norm --> ReLU --> Dropout)*2 + output --> (FC:1042 --> FC:3)
+ sign shows skip connections
* To avoid overfitting use: dropout, batch normalization, weight regularization, increase size of training set for by performing some data augmentations 

In [ ]:
class RNNModel(torch.nn.Module):
    
    def __init__(self, input_dim, class_num, initrange=0.5):
        super(RNNModel, self).__init__()
        self.input = input_dim
        self.hidden_size = 24
        self.rnn = nn.RNN(self.input, self.hidden_size, num_layers=5, dropout=0.3 , batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size*5,160)
        self.bn1 = torch.nn.BatchNorm1d(160)
        self.fc2 = torch.nn.Linear(160,160)
        self.bn2 = torch.nn.BatchNorm1d(160)
        self.fc3 = torch.nn.Linear(160,80)
        self.bn3 = torch.nn.BatchNorm1d(80)
        self.fc4 = torch.nn.Linear(80, class_num)
        self.init_weights(initrange)
        self.class_num = class_num
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
    
    
    def forward(self, _input, batch_size):
        
        hidden_cell = torch.randn(5, batch_size, self.hidden_size, dtype=torch.float).to(device)
        
        rnn_feat, newh_cell= self.rnn(_input, hidden_cell)
        rnn_feat = rnn_feat.reshape((-1,rnn_feat.shape[1]*rnn_feat.shape[2]))
        
        _fc1 = self.bn1(self.fc1(rnn_feat))
        #_fc1 = F.dropout(F.relu(_fc1), p=0.3)
        _fc1 = F.relu(_fc1)
        
        _fc2 = self.bn2(self.fc2(_fc1))
        _fc2 = F.relu(_fc2)
        
        _fc3 = self.bn3(self.fc3(_fc2))
        _fc3 = F.relu(_fc3)
        
        _fc4 = self.fc4(_fc3)
        output = F.softmax(_fc4, dim=1)
        
        return output

In [ ]:
class GRUModel(torch.nn.Module):
    
    def __init__(self, input_dim, class_num, initrange=0.5):
        super(GRUModel, self).__init__()
        self.input = input_dim
        self.hidden_size = 24
        self.rnn = nn.GRU(self.input, self.hidden_size, num_layers=5, dropout=0.3 , batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size*5,160)
        self.bn1 = torch.nn.BatchNorm1d(160)
        self.fc2 = torch.nn.Linear(160,160)
        self.bn2 = torch.nn.BatchNorm1d(160)
        self.fc3 = torch.nn.Linear(160,80)
        self.bn3 = torch.nn.BatchNorm1d(80)
        self.fc4 = torch.nn.Linear(80, class_num)
        self.init_weights(initrange)
        self.class_num = class_num
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
    
    
    def forward(self, _input, batch_size):
        
        hidden_cell = torch.randn(5, batch_size, self.hidden_size, dtype=torch.float).to(device)
        
        rnn_feat, newh_cell= self.rnn(_input, hidden_cell)
        rnn_feat = rnn_feat.reshape((-1,rnn_feat.shape[1]*rnn_feat.shape[2]))
        
        _fc1 = self.bn1(self.fc1(rnn_feat))
        #_fc1 = F.dropout(F.relu(_fc1), p=0.3)
        _fc1 = F.relu(_fc1)
        
        _fc2 = self.bn2(self.fc2(_fc1))
        _fc2 = F.relu(_fc2)
        
        _fc3 = self.bn3(self.fc3(_fc2))
        _fc3 = F.relu(_fc3)
        
        _fc4 = self.fc4(_fc3)
        output = F.softmax(_fc4, dim=1)
        
        return output

In [ ]:
class LSTMModel(torch.nn.Module):
    
    def __init__(self, input_dim, class_num,seq_len=10, initrange=0.5):
        super(LSTMModel, self).__init__()
        self.input = input_dim
        self.hidden_size = 256
        self.rnn = nn.LSTM(self.input, self.hidden_size, num_layers=5, dropout=0.5 , batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size*seq_len,512)
        self.bn1 = torch.nn.BatchNorm1d(512)
        self.fc2 = torch.nn.Linear(512,160)
        self.bn2 = torch.nn.BatchNorm1d(160)
        #self.fc3 = torch.nn.Linear(160,80)
        #self.bn3 = torch.nn.BatchNorm1d(80)
        self.fc4 = torch.nn.Linear(160, class_num)
        self.init_weights(initrange)
        self.class_num = class_num
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        #self.fc3.weight.data.uniform_(-initrange, initrange)
        #self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
    
    
    def forward(self, _input, batch_size):
        
        h_cell = torch.randn(5, batch_size, self.hidden_size, dtype=torch.float).to(device)
        c_cell = torch.randn(5, batch_size, self.hidden_size, dtype=torch.float).to(device)
        rnn_feat, (newh_cell, newc_cell) = self.rnn(_input, (h_cell, c_cell))
        rnn_feat = rnn_feat.reshape((-1,rnn_feat.shape[1]*rnn_feat.shape[2]))
        
        _fc1 =self.bn1(self.fc1(rnn_feat))
        #_fc1 = F.dropout(F.relu(_fc1), p=0.3)
        _fc1 = F.relu(_fc1)
        
        _fc2 = self.bn2(self.fc2(_fc1))
        _fc2 = F.relu(_fc2)
        
        #_fc3 = self.bn3(self.fc3(_fc2))
        #_fc3 = F.relu(_fc3)
        
        _fc4 = self.fc4(_fc2)
        output = F.softmax(_fc4, dim=1)
        
        return output

In [ ]:
class DNN_Single(torch.nn.Module):
    
    def __init__(self, input_dim, class_num, initrange=0.5):
        
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.fc3 = torch.nn.Linear(64,16)
        self.bn3 = torch.nn.BatchNorm1d(16)
        self.fc4 = torch.nn.Linear(16, class_num)
        self.class_num = class_num
        self.init_weights(initrange)
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
    
    def forward(self, _input):
        
        _fc1 = F.relu(self.fc1(_input))
        
        _bn1 = self.bn1(_fc1)
        
        _fc2 = F.relu(self.fc2(_bn1))
        _bn2 = self.bn2(_fc2)
        
        _fc3 = F.relu(self.fc3(_bn2))
        _bn3 = self.bn3(_fc3)
        
        _fc4 = self.fc4(_bn3)
        
        output = F.softmax(_fc4, dim=1)
        
        return output

In [ ]:
def prepare_data(reshape=False, seq_len=10, bs=8, n_test = 0.35, n_valid=0.2):
    
    # load pose dataset
    dataset = SinglePoseDataset(n_frames=seq_len)
    targets = dataset.y
    
    # reshape from N,10,34 to N,340
    if reshape:
        dataset.reshape_features()
    
    # calculate split size
    #train,  valid, test = dataset.get_splits()
    
    #stratified train test split
    train_idx, valid_idx = train_test_split(np.arange(len(targets)), test_size=0.35, stratify=targets)
    
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    train_dl  = DataLoader(dataset, batch_size=bs, sampler=train_sampler, drop_last=True)
    valid_dl  = DataLoader(dataset, batch_size=512, sampler=valid_sampler, drop_last=True)
    
    # prepare data loaders
    #train_dl =  DataLoader(train, batch_size=bs, shuffle=True, drop_last=True)
    #valid_dl  = DataLoader(valid, batch_size=512, shuffle=False, drop_last=True)
    #test_dl  =  DataLoader(test, batch_size=1024, shuffle=False, drop_last=True)
    
    return {'train':train_dl, 'valid':valid_dl}, {'train':len(train_sampler), 'valid':len(valid_sampler)}

In [ ]:
def save_model(model, optimizer, loss, acc, epoch, save_path):
    
    base_dir = os.path.basename(os.getcwd())
    
    if base_dir =='train':
        parent_dir = os.path.dirname(os.getcwd())
        os.chdir(parent_dir)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    
    print('SAVING EPOCH %d'%epoch)
    filename = 'epoch_%d'%epoch + '_loss_%f.pth'%loss
    SAVE_FILE = os.path.join(save_path,filename)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'acc':  acc,
            }, SAVE_FILE)

In [ ]:
def train_model(model, dataloaders, dataset_sizes, num_epochs=3000):
    
    
    history = defaultdict(list)
    
    #get train and val dataloaders
    #dataloaders, dataset_sizes = prepare_data()

    # define the optimization
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=300, gamma=0.1)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=5, factor=0.1,verbose=True)
    
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = -1
    conf_train = 0.0
    conf_valid = 0.0
    # enumerate over epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train() # St model to training mode
            
            else:
                model.eval() # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0.0
            # Initialize the prediction and label lists(tensors)
            pred_tensor  = torch.zeros(0,dtype=torch.long, device='cpu')
            class_tensor = torch.zeros(0,dtype=torch.long, device='cpu')
            
            # enumerate over mini_batch
            for i, (inputs, targets)  in enumerate(dataloaders[phase]):
                inputs  = inputs.to(device)
                targets = targets.to(device)
                
                # clear the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train pahse
                with torch.set_grad_enabled(phase=='train'):
                    
                    # compute model outputs
                    outputs = model(inputs, inputs.shape[0])
                    
                    # calculate outputs
                    _, preds = torch.max(outputs, dim=1)
                    
                    # calculate the loss
                    loss = criterion(outputs, targets)
                    
                    # backward + optimize only ig in training phase
                    if phase == 'train':
                        # calculate gradient
                        loss.backward()
                    
                        # update model weights
                        optimizer.step()
                
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==targets)
                
                # Append batch prediction results
                pred_tensor = torch.cat([pred_tensor,preds.view(-1).cpu()])
                class_tensor  = torch.cat([class_tensor,targets.view(-1).cpu()])
                
            
            #epoch loss and accuracy
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc  = running_corrects.double() / dataset_sizes[phase]
            history[phase].append((epoch_loss, epoch_acc, ))
            
            if phase=='valid':
                scheduler.step()

            # Confusion matrix
            conf_mat = confusion_matrix(class_tensor.numpy(), pred_tensor.numpy())
            # Per-class accuracy
            #per_class_accuracy = np.round(100*conf_mat.diagonal()/conf_mat.sum(1),4)
            #Precision, Recall, F1_Score
            precision = precision_score(class_tensor.numpy(), pred_tensor.numpy(), average=None)
            recall = recall_score(class_tensor.numpy(), pred_tensor.numpy(), average=None)
            f_score = f1_score(class_tensor.numpy(), pred_tensor.numpy(), average=None)
            
            print('{} : Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            #print('{} : Confusion Matrix: {}'.format(phase, conf_mat))
            #print('{} : Precision per class: {}'.format(phase, np.round(precision,4)))
            #print('{} : Recall per class: {}'.format(phase, np.round(recall,4)))
            print('{} : F1_Score per class: {}'.format(phase, np.round(f_score,4)))
            print()
            
            if phase== 'valid' and loss_< epoch_loss:
                epoch_ = epoch
                loss_  = epoch_loss
                conf_valid = conf_mat
                best_model_wts = copy.deepcopy(model.state_dict())
                save_model(model, optimizer, loss_, epoch_acc, epoch_, save_path=r'checkpoints_fallmodel/act_fclstm_15')
            
            if phase== 'train' and epoch_acc>best_acc:
                conf_train = conf_mat
               
                
        print()
    
    time_elapsed = time.time() - since

    
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed //60, time_elapsed %60))
    print('Best val Acc: {:4f}'.format(epoch_acc))
    
    # load best model weights
    #model.load_state_dict(best_model_wts)
    #save_model(model, optimizer, epoch_loss, best_acc, epoch, save_path=r'checkpoints_fallmodel/act_fclstm_15')
    
    return history, model, conf_train, conf_valid

In [ ]:
#RNN_model = RNNModel(input_dim=24, class_num=3).to(device)

In [ ]:
#get test dataloaders
#dataloaders, dataset_sizes = prepare_data(batch_size=16)
#history, model, conf_train, conf_valid = train_model(RNN_model, dataloaders, dataset_sizes,num_epochs=3000)

In [ ]:
#GRU_model = GRUModel(input_dim=24, class_num=3).to(device)

In [ ]:
#get test dataloaders
#dataloaders, dataset_sizes = prepare_data(batch_size=16)
#history, model, conf_train, conf_valid = train_model(GRU_model, dataloaders, dataset_sizes,num_epochs=3000)

In [ ]:
LSTM_model = LSTMModel(input_dim=24, class_num=3, seq_len=15).to(device)
total_params = sum(p.numel() for p in LSTM_model.parameters() if p.requires_grad)
print(total_params)

In [ ]:
#get test dataloaders
dataloaders, dataset_sizes = prepare_data(bs=512, seq_len=15)

In [ ]:
history, model, conf_train, conf_valid = train_model(LSTM_model, dataloaders, dataset_sizes,num_epochs=2000)

In [ ]:
train_acc  = []
train_loss = []
val_acc =  []
val_loss = []

for train_item, val_item in zip(history['train'],history['valid']):
    
    val_loss.append(val_item.__getitem__(0))
    val_acc.append(val_item.__getitem__(1).cpu().detach().numpy())
    
    train_loss.append(train_item.__getitem__(0))
    train_acc.append(train_item.__getitem__(1).cpu().detach().numpy())

In [ ]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
num_epochs=range(2000)
plt.figure(figsize=(12, 6))
plt.plot(num_epochs, train_acc, label='Training Accuracy')
plt.plot(num_epochs, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy', fontsize=18)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('plots/lstm_seq10_acc.jpg')
plt.show()

In [ ]:
num_epochs=range(2000)
plt.figure(figsize=(12, 6))
plt.plot(num_epochs, train_loss, label='Training Loss')
plt.plot(num_epochs, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('plots/lstm_seq10_loss.jpg')
plt.show()

In [ ]:
#set the size of figure 542129345
plt.figure(figsize=(8,8))
#normalize each column (class) with total datapoints in that column  
conf_train = conf_train.astype('float')/conf_train.sum(axis=1)*100
#plot confusion matrix 
p=sns.heatmap(conf_train, xticklabels=['Fall','Stand','Tie'], yticklabels=['Fall','Stand','Tie'],
              cbar=False, annot=True, cmap='coolwarm',robust=True, fmt='.1f',annot_kws={'size':20})
plt.title('Training matrix: Actual labels Vs Predicted labels')
plt.savefig('plots/lstm_seq10_train_cf.png')

In [ ]:
#set the size of figure 
plt.figure(figsize=(8,8))
#normalize each column (class) with total datapoints in that column  
conf_valid = conf_valid.astype('float')/conf_valid.sum(axis=1)*100
#plot confusion matrix 
p=sns.heatmap(conf_valid, xticklabels=['Fall','Stand','Tie'], yticklabels=['Fall','Stand','Tie'],
              cbar=False, annot=True, cmap='coolwarm',robust=True, fmt='.1f',annot_kws={'size':20})
plt.title('Validation matrix: Actual labels vs Predicted labels')
plt.savefig('plots/lstm_seq10_valid_cf.png')

In [ ]:
def evaluate_model(model, dataloaders, dataset_sizes):
    #defaultdict to save list in dict
    history = defaultdict(list)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    model.eval() # Set model to evaluate mode
    phase = 'test'            
    running_loss = 0.0
    running_corrects = 0.0
    # Initialize the prediction and label lists(tensors)
    pred_tensor  = torch.zeros(0,dtype=torch.long, device='cpu')
    class_tensor = torch.zeros(0,dtype=torch.long, device='cpu')

    # enumerate over mini_batch
    for i, (inputs, targets)  in enumerate(dataloaders['test']):
        inputs  = inputs.to(device)
        targets = targets.to(device)

        # compute model outputs
        outputs = model(inputs, inputs.shape[0])

        # calculate outputs
        _, preds = torch.max(outputs, dim=1)

        # calculate the loss
        loss = criterion(outputs, targets)

        #statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds==targets)

        # Append batch prediction results
        pred_tensor = torch.cat([pred_tensor,preds.view(-1).cpu()])
        class_tensor  = torch.cat([class_tensor,targets.view(-1).cpu()])
                
    #epoch loss and accuracy
    loss = running_loss / dataset_sizes['test'].__len__()
    acc  = running_corrects.double() / dataset_sizes['test'].__len__()
    history['test'].append((loss,acc))

    # Confusion matrix
    conf_mat = confusion_matrix(class_tensor.numpy(), pred_tensor.numpy())
    # Per-class accuracy
    per_class_accuracy = np.round(100*conf_mat.diagonal()/conf_mat.sum(1),4)
    #Precision, Recall, F1_Score
    precision = precision_score(class_tensor.numpy(), pred_tensor.numpy(), average='micro')
    recall = recall_score(class_tensor.numpy(), pred_tensor.numpy(), average='micro')
    f_score = f1_score(class_tensor.numpy(), pred_tensor.numpy(), average='micro')

    #print('{} : Loss: {:.4f}, Acc: {:.4f}'.format(phase, loss, acc))
    #print('{} : Confusion Matrix: {}, Per_class_accuracy: {}'.format(phase, conf_mat, per_class_accuracy))
    print('{} : Precision: {:.4f}, Recall: {:.4f}, F1_Score: {:.4f}'.format(phase, precision, recall, f_score))
    print()
            
    #set the size of figure 
    plt.figure(figsize=(8,8))
    #normalize each column (class) with total datapoints in that column  
    conf_mat = conf_mat.astype('float')/conf_mat.sum(axis=1)*100
    #plot confusion matrix 
    p=sns.heatmap(conf_mat, xticklabels=['Fall','Stand','Tie'], yticklabels=['Fall','Stand','Tie'],
              cbar=False, annot=True, cmap='coolwarm',robust=True, fmt='.1f',annot_kws={'size':20})
    plt.title('Test matrix: Actual labels Vs Predicted labels')
    plt.savefig('plots/lstm_seq10_test_cf.png')

In [ ]:
evaluate_model(model, dataloaders, dataset_sizes)